In [13]:
import os
if(not os.getcwd().endswith("crchum")):
    os.chdir("..")
print(os.getcwd())
env = ! cat .env
try : 
    DEVICE = [_2.replace("\"", "") for (_1, _2) in [e.split("=") for e in env] if _1 == "DEVICE"][0]
except:
    print("Set the device in .env file")
print("Device : ", DEVICE)
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

/Users/antoinemagron/Documents/EPFL/PDM/crchum
Device :  mps


In [3]:
from transformers import (AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM)

/opt/homebrew/Caskroom/miniconda/base/envs/nlp1/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/nlp1/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/nlp1/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [6]:
# MODEL = "epfl-llm/meditron-7b" ## meditron-7B plain
# MODEL = "meta-llama/Llama-2-7b-hf" ## LLAMA-7B rough
# MODEL = "meta-llama/Llama-2-7b-chat-hf" ## LLAMA-7B chat
MODEL = "malhajar/meditron-7b-chat" ## Meditron-chat
tok = AutoTokenizer.from_pretrained(MODEL)
meditron = AutoModelForCausalLM.from_pretrained(MODEL)

KeyboardInterrupt: 

In [14]:
import torch
if torch.backends.mps.is_available():
    pt_device = torch.device(DEVICE)
    x = torch.ones(1, device=pt_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [6]:
import json
import numpy as np
with open("docs/benchmarks/mcq40/processed.json", "r") as f:
    mcq = json.load(f)

In [7]:
prompt = """
    Question : {mcq[0][question]}
    Answer : {mcq[0][answer]}

    Question : {mcq[1][question]}
    Answer : {mcq[1][answer]}
    
    Question : {mcq[2][question]}
    Answer : {mcq[2][answer]}
    
    Question : {mcq[3][question]}
    Answer :""".format(**{"mcq":mcq})
print(prompt)


    Question : Which of the following is the major role of the neurologist after a diagnosis of psychogenic nonepileptic events?
        A. appropriately weaning antiepileptic drugs
        B. describing events in ambiguous terms
        C. ensuring that patients continue to refrain from driving a motor vehicle
        D. ordering follow-up EEG in 6 to 12 months
        E. prescribing a selective serotonin reuptake inhibitor or other antidepressant
    Answer : A

    Question : Deep brain stimulation for drug-resistant epilepsy targets which of the following structures?
        A. anterior nucleus of the thalamus
        B. centromedian nucleus of the thalamus
        C. cingulate gyrus
        D. hippocampus
        E. internal segment of the globus pallidus
    Answer : A
    
    Question : According to the International League Against Epilepsy’s 2010 revision of its seizure classification system, which of the following factors is most helpful in initially focusing the scope of di

In [35]:
def format_question(q, a):
    sysmsg = """You are a student passing a neurology exam. 
    You will be given a question with 5 options and you will need to give the letter of the corresponding correct option."""
    # return f"<|im_start|> system\n{sysmsg}<|im_end|>\n <|im_start|> user\n{q}<|im_end|>\n <|im_start|> assistant\n{a}"""
    return f"-system\n{sysmsg}\n-user\n{q}\n-assistant:\n{a}"""




questions = [
    ["""Question : Which of the following is the major role of the neurologist after a diagnosis of psychogenic nonepileptic events?,
        A. appropriately weaning antiepileptic drugs
        B. describing events in ambiguous terms
        C. ensuring that patients continue to refrain from driving a motor vehicle
        D. ordering follow-up EEG in 6 to 12 months
        E. prescribing a selective serotonin reuptake inhibitor or other antidepressant""",
    """Answer : A"""],

    ["""Question : Deep brain stimulation for drug-resistant epilepsy targets which of the following structures?
        A. anterior nucleus of the thalamus
        B. centromedian nucleus of the thalamus
        C. cingulate gyrus
        D. hippocampus
        E. internal segment of the globus pallidus""",
    """Answer : A"""],
    
    ["""Question : According to the International League Against Epilepsy’s 2010 revision of its seizure classification system, which of the following factors is most helpful in initially focusing the scope of diagnostic possibilities for specific electroclinical syndromes?
        A. age at onset
        B. family seizure history
        C. frequency of spells
        D. history of toxic exposure
        E. severity of the disorder""",
    """Answer : A"""
    ],
    
    ["""Question : Which of the following conditions would likely preclude the consideration of epilepsy surgery in a patient?
        A. dominant hemisphere focus
        B. low-grade tumor
        C. mental retardation
        D. primary generalized epilepsy
        E. prior vagus nerve stimulator use""",
    """Answer :"""
    ]
]


full_prompt = "\n".join([format_question(q, a) for q, a in questions])
print(full_prompt)

-system
You are a student passing a neurology exam. 
    You will be given a question with 5 options and you will need to give the letter of the corresponding correct option.
-user
Question : Which of the following is the major role of the neurologist after a diagnosis of psychogenic nonepileptic events?,
        A. appropriately weaning antiepileptic drugs
        B. describing events in ambiguous terms
        C. ensuring that patients continue to refrain from driving a motor vehicle
        D. ordering follow-up EEG in 6 to 12 months
        E. prescribing a selective serotonin reuptake inhibitor or other antidepressant
-assistant:
Answer : A
-system
You are a student passing a neurology exam. 
    You will be given a question with 5 options and you will need to give the letter of the corresponding correct option.
-user
Question : Deep brain stimulation for drug-resistant epilepsy targets which of the following structures?
        A. anterior nucleus of the thalamus
        B. centr

In [30]:
allans = []
allk = [1, 10]
allp = [.5, .95]
temp = [.5, 1, 1.5]
for k in allk:
    for p in allp:
        for t in temp:
            print(f"Computing for k={k} p={p} t={t}", end="\r")
            meditron = meditron.to(mps_device)
            answer = meditron.generate(
                **tok(full_prompt, return_tensors="pt").to(mps_device), 
                temperature=t, 
                max_new_tokens=1, 
                top_k=k,
                top_p=p
            )
            allans.append(
                tok.batch_decode(answer)[0]
            )

/opt/homebrew/Caskroom/miniconda/base/envs/nlp1/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/nlp1/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/nlp1/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnin

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


/opt/homebrew/Caskroom/miniconda/base/envs/nlp1/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


/opt/homebrew/Caskroom/miniconda/base/envs/nlp1/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [32]:
params = []
for k in allk:
    for p in allp:
        for t in temp:
            params.append((k, p, t))

for (k, p, t), ans in zip(params, allans):
    print(f"For k={k} p={p} t={p}")
    print(ans.split("\n")[-1])

For k=1 p=0.5 t=0.5
Answer : -
For k=1 p=0.5 t=0.5
Answer : fine
For k=1 p=0.5 t=0.5
Answer :aucoup
For k=1 p=0.95 t=0.95
Answer : -
For k=1 p=0.95 t=0.95
Answer :-
For k=1 p=0.95 t=0.95
Answer : oft
For k=10 p=0.5 t=0.5
Answer : oft
For k=10 p=0.5 t=0.5
Answer :oux
For k=10 p=0.5 t=0.5
Answer :aucoup
For k=10 p=0.95 t=0.95
Answer :aucoup
For k=10 p=0.95 t=0.95
Answer :ands
For k=10 p=0.95 t=0.95
Answer :aucoup


In [36]:
t = .95
p = .95
k = 1
answer = meditron.generate(
                **tok(full_prompt, return_tensors="pt").to(mps_device), 
                temperature=t, 
                max_new_tokens=10, 
                top_k=k,
                top_p=p
            )

stans = tok.batch_decode(answer)[0]
print(stans)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> -system
You are a student passing a neurology exam. 
    You will be given a question with 5 options and you will need to give the letter of the corresponding correct option.
-user
Question : Which of the following is the major role of the neurologist after a diagnosis of psychogenic nonepileptic events?,
        A. appropriately weaning antiepileptic drugs
        B. describing events in ambiguous terms
        C. ensuring that patients continue to refrain from driving a motor vehicle
        D. ordering follow-up EEG in 6 to 12 months
        E. prescribing a selective serotonin reuptake inhibitor or other antidepressant
-assistant:
Answer : A
-system
You are a student passing a neurology exam. 
    You will be given a question with 5 options and you will need to give the letter of the corresponding correct option.
-user
Question : Deep brain stimulation for drug-resistant epilepsy targets which of the following structures?
        A. anterior nucleus of the thalamus
        B. c

<ins>Generation time</ins> : 1min49 for one answer 